In [1]:
import os 
%pwd

'/home/towet/Desktop/OpenProjects/Malaria-Cell-Histopathology-Classification-Transfer-Learning/research'

In [2]:
os.chdir('../')

In [3]:
%pwd

'/home/towet/Desktop/OpenProjects/Malaria-Cell-Histopathology-Classification-Transfer-Learning'

In [4]:
from dataclasses import dataclass 
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path 
    model_path: Path 
    dataset: Path 
    epochs: int 
    loss: str 
    optimizer: str 
    base_model: str
    batch_size: int 
    metrics: list 
    img_size: tuple
   
    

In [5]:
from MalariaCellHistopathologyClassification.constants import *
from MalariaCellHistopathologyClassification.utils.common import read_yaml, create_directories

2024-03-05 13:49:15.541593: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-05 13:49:15.608705: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-05 13:49:15.608774: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-05 13:49:15.610531: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-05 13:49:15.619679: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-05 13:49:16.851366: W tensorflow/compiler/tf2tensorrt/utils/py_utils.

In [6]:
class ConfigurationManager:
    def __init__(self, 
                 config_file_path = CONFIG_FILE_PATH,
                 param_file_path = PARAMS_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.param = read_yaml(param_file_path)
        
        create_directories([self.config.artifacts_root])
        
    def get_training_config(self) -> TrainingConfig:
        config = self.config.training 
        param = self.param
        create_directories([config.root_dir])
        dataset = "artifacts/data_ingestion/dataset/"
        training_config = TrainingConfig(
            root_dir=config.root_dir,
            model_path=config.model_path,
            epochs=param.EPOCH,
            base_model=param.BASE_MODEL,
            img_size=param.IMG_SIZE,
            batch_size=param.BATCH_SIZE,
            optimizer=param.OPTIMIZER,
            loss=param.LOSS,
            metrics=param.METRICS,
            dataset=str(dataset)
        )
        return training_config

In [7]:
import tensorflow as tf 
import numpy as np 
from MalariaCellHistopathologyClassification.utils.common import load__process_dataset, save_plot

In [12]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config 
        
    def train(self):
        train_ds, val_ds, _, _ = load__process_dataset(self.config.dataset,
                                                                 self.config.batch_size, 
                                                                 self.config.img_size)
        preprocess_input = tf.keras.applications.vgg19.preprocess_input
        IMG_SHAPE = (224,224,3)
        base_model = tf.keras.applications.VGG19(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')
        image_batch, label_batch = next(iter(train_ds))
        feature_batch = base_model(image_batch)
        
        for layer in base_model.layers:
            layer.tranable=False
            
        global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
        feature_batch_average = global_average_layer(feature_batch)
        prediction_layer = tf.keras.layers.Dense(1)
        prediction_batch = prediction_layer(feature_batch_average)
        
        inputs = tf.keras.Input(shape=(224, 224, 3))
        x = preprocess_input(inputs)
        x = base_model(x, training=False)
        x = global_average_layer(x)
        x = tf.keras.layers.Dropout(0.2)(x)
        outputs = prediction_layer(x)
        model = tf.keras.Model(inputs, outputs)
        
        
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=[tf.keras.metrics.BinaryAccuracy(threshold=0, name='accuracy')])
        
        history = model.fit(train_ds,
                    epochs=self.config.epochs,
                    steps_per_epoch=len(train_ds)/32,
                    validation_data=val_ds,
                    validation_steps=len(val_ds)/32,
                   verbose=1

                    )
        model.save(self.config.model_path)   
        save_plot(history, "src/MalariaCellHistopathologyClassification/models/plots.py/history.png") 
        

In [13]:
try:
    config = ConfigurationManager()
    train_config = config.get_training_config()
    training = Training(config=train_config)
    training.train()
except Exception as e:
    raise e

[2024-03-05 13:56:16,718: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-03-05 13:56:16,915: INFO: common: yaml file: params.yaml loaded successfully]
[2024-03-05 13:56:16,940: INFO: common: created directory at: artifacts]
[2024-03-05 13:56:16,948: INFO: common: created directory at: artifacts/training]
Found 27558 files belonging to 2 classes.
Using 22047 files for training.
Found 27558 files belonging to 2 classes.
Using 5511 files for validation.
